In [10]:
# Step 1: Import necessary libraries
import os
import pandas as pd
import subprocess

os.chdir('/mnt/belinda_local/ruth/data/PrimateAI-3D_tests/')

# Step 2: Load your genetic information matrix and phenotype dataframe
genetic_info_matrix = pd.read_csv('genetic_info_matrix.csv')  # Replace with your actual file path
phenotype_df = pd.read_csv('phenotype_data.csv')  # Replace with your actual file path

# Display the first few rows of the genetic information and phenotype data
print("Genetic Information Matrix:")
print(genetic_info_matrix.head())
print("\nPhenotype DataFrame:")
print(phenotype_df.head())

# Step 3: Prepare your input for VEP
# Save the genetic matrix in VCF format
def save_to_vcf(genetic_info_matrix, vcf_file):
    with open(vcf_file, 'w') as f:
        # Write the VCF header
        f.write('##fileformat=VCFv4.2\n')
        f.write('##INFO=<ID=DP,Number=1,Type=Integer,Description="Depth of Coverage">\n')
        f.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n')
        
        # Write the variant data
        for index, row in genetic_info_matrix.iterrows():
            f.write(f"{row['chromosome']}\t{row['position']}\t.\t{row['ref_allele']}\t{row['alt_allele']}\t.\t.\tDP=.\n")

# Save genetic information to a VCF file
save_to_vcf(genetic_info_matrix, 'input_variants.vcf')

# Step 4: Annotate the variants using VEP
# This command assumes VEP is installed and available in your PATH
# vep_command = [
#     'vep', 
#     '-i', 'input_variants.vcf', 
#     '--cache', 
#     '--dir_cache', 'path/to/vep/cache',  # Replace with your VEP cache path
#     '--format', 'vcf', 
#     '--assembly', 'GRCh38', 
#     '--output_file', 'annotated_variants.vcf'
# ]
# subprocess.run(vep_command)
vep_output_df = pd.read_csv('annotated_variants.vcf', sep='\t')  # Replace with actual path
print("\nVEP Output:")
print(vep_output_df.head())

# Step 5: Run PrimateAI-3D to calculate pathogenicity scores
# Ensure you have PrimateAI-3D set up and accessible
primate_ai_command = [
    'python', '/mnt/belinda_local/ruth/home/PrimateAI-3D/primateAI-3D/worker.py',  # Replace with the path to your PrimateAI.py script
    '--input', 'annotated_variants.vcf', 
    '--output', 'pathogenicity_scores.txt'
]
print(primate_ai_command)
# subprocess.run(primate_ai_command) # Some dependencies seem to be lacking

# Useful program may also be inside rare_polygenic/example_data/run.sh - but rvPRS is not found
# Run setup.py inside rare_polygenic may be necessary - but some dependencies are lacking

# # Step 6: Load the pathogenicity scores
# pathogenicity_scores = pd.read_csv('pathogenicity_scores.txt', sep='\t')  # Adjust separator if needed
# print("\nPathogenicity Scores:")
# print(pathogenicity_scores.head())

# # Step 7: Merge pathogenicity scores with phenotype data
# # Assuming you have a common identifier for merging
# merged_data = phenotype_df.merge(pathogenicity_scores, on='sample_id')  # Adjust as necessary
# print("\nMerged Data:")
# print(merged_data.head())

# # Step 8: Perform statistical analysis (e.g., logistic regression)
# import statsmodels.api as sm

# # Define the independent variables (pathogenicity scores) and dependent variable (phenotype)
# X = merged_data[['pathogenicity_score']]  # Replace with actual score column name
# y = merged_data['phenotype']  # Replace with actual phenotype column name

# # Add constant for the intercept
# X = sm.add_constant(X)

# # Fit logistic regression model
# model = sm.Logit(y, X)
# result = model.fit()

# # Step 9: Print the results summary
# print("\nLogistic Regression Results:")
# print(result.summary())


Genetic Information Matrix:
   chromosome  position ref_allele alt_allele
0           1       101          A          C
1           1       102          G          A
2           2       201          C          G
3           2       202          T          C
4           3       301          A          T

Phenotype DataFrame:
  sample_id  phenotype
0   sample1          0
1   sample2          1
2   sample3          0
3   sample4          1
4   sample5          0

VEP Output:
  Uploaded_variation Location Allele   Gene       Feature         Consequence  \
0          1_101_A/C    1:101      C  GENE1  ENST000002.1    missense_variant   
1          1_102_G/A    1:102      A  GENE2  ENST000003.1  synonymous_variant   
2          2_201_C/G    2:201      G  GENE3  ENST000004.1    missense_variant   
3          2_202_T/C    2:202      C  GENE4  ENST000005.1      intron_variant   
4          3_301_A/T    3:301      T  GENE5  ENST000006.1    missense_variant   

   Protein_position Amino_acids   Co